Define libraries imports and the dictonary containing the params to execute the .smt2 files with the 2 chosen solvers: Z3 and CVC5.

In [ ]:
import os
import sys
import re
import time
import json
import argparse
import subprocess

ROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), "../.."))
sys.path.insert(0, ROOT)

from solution_checker import check_solution

# ==== GLOBAL CONFIG ====
GLOBAL_TIMEOUT = 300.0

SOLVERS = {
    "z3": {
        "cmd":     "z3",
        "timeout": lambda t: ["-T:" + str(int(t))],
        "model":   []
    },
    "cvc5": {
    "cmd":     "cvc5",
    "timeout": lambda t: ["--tlimit=" + str(int(t*1000))],
    "model":   ["--produce-models"]
    }

}

Function to run the solver on the "smtfile". The GLOBAL_TIMEOUT is passed, so it can check if it goes in timeout.

In [ ]:
def run_solver(name, smtfile, timeout_s = GLOBAL_TIMEOUT):

    s = SOLVERS[name]
    cmd = [s["cmd"]] + s["timeout"](timeout_s) + s["model"] + [smtfile]
    start = time.time()
    proc = subprocess.run(cmd, capture_output=True, text=True)
    return proc.stdout, time.time() - start

This function is used at the beginning of the optimization phase to compute the starting value of the imbalance k. From this value we then start the binary search-

Input:
*   S: tensor representing rb.
*   t: a team.

Output: imbalance between home and away games for team t.




In [ ]:
def imbalance_of(S, t):

    P, W = len(S), len(S[0])
    home = sum(1 for p in range(P) for w in range(W) if S[p][w][0] == t)
    away = sum(1 for p in range(P) for w in range(W) if S[p][w][1] == t)
    return abs(home - away)

### SMT-LIB Generation: Satisfiability Phase

This function starts the **first phase** of the pipeline: generating a `.smt2` file that models the STS problem as a satisfiability query. The goal is to encode all **decision variables** and **constraints** line-by-line into SMT-LIB format.

We first compute the round-robin matrix `rb` using the Giulio2 method (God bless his soul). This matrix defines, for each period and week, the candidate match (team pair) to be scheduled.

The key object is the `lines` list, which accumulates all SMT-LIB commands:

* It begins by setting the logic (`QF_LIA`) and enabling model generation.
* Then, all **decision variables** are declared:

  * `index_{p,w,t}`: Boolean selector for whether pair `t` is chosen in slot `(p,w)`.
  * `home_{p,w}`, `away_{p,w}`: Integer variables for the assigned teams.

Next, we add the constraints:

* **Slot uniqueness**: each slot `(p,w)` must select exactly one pair.
* **Pair uniqueness**: each pair `t` must appear exactly once per week.
* **Period limit**: no team should play more than twice in the same period across all weeks.

We then **bind the values** of `home_{p,w}` and `away_{p,w}` based on which `index_{p,w,t}` is selected.

To **break symmetries** and reduce equivalent solutions, we fix the first pair in the first slot.

Finally, we append `(check-sat)` and `(get-model)` to request a solution, and write everything to the `.smt2` file at the appropriate path.

In [ ]:
# ==== SATISFABILITY ====

def satisfability_generate_smt2(n, fname):

    THIS_DIR = os.path.dirname(__file__)
    smt_folder = os.path.join(THIS_DIR, "smt")
    os.makedirs(smt_folder, exist_ok=True)

    P, W = n // 2, n - 1
    rb = [] # rb[p][w] holds the (home, awmatrixay) pair assigned to index t in slot (p,w)
    for p in range(P):
        week = []
        for w in range(W):
            if p == 0:
                pair = (n, w + 1)
            else:
                pair = (((p + w) % (n - 1)) + 1, ((n - p + w - 1) % (n - 1)) + 1)
            week.append(tuple(sorted(pair)))
        rb.append(week)

    # Initialize SMT-LIB script: set logic and model production
    lines = [
        "(set-logic QF_LIA)",
        "(set-option :produce-models true)"
    ]

    # Declare decision variables:
    # For each slot: P periods x W weeks
    # - index_{p,w,t} : Boolean selector for pair t in slot (p,w)
    # - home_{p,w}, away_{p,w} : Integers for the assigned teams
    for p in range(P):
        for w in range(W):
            for t in range(P):
                lines.append(f"(declare-fun index_{p}_{w}_{t} () Bool)")
            lines.append(f"(declare-fun home_{p}_{w} () Int)")
            lines.append(f"(declare-fun away_{p}_{w} () Int)")

    # Each slot must select exactly one pair:
    # Sum over all index_{p,w,t} must be 1
    for p in range(P):
        for w in range(W):
            terms = " ".join(f"(ite index_{p}_{w}_{t} 1 0)" for t in range(P))
            lines.append(f"(assert (= (+ {terms}) 1))")

    # Each pair must be used exactly once per week:
    # Sum over all slots using pair t must be 1
    for t in range(P):
        for w in range(W):
            terms = " ".join(f"(ite index_{p}_{w}_{t} 1 0)" for p in range(P))
            lines.append(f"(assert (= (+ {terms}) 1))")

    # For each slot, if index_{p,w,t} is true, bind home/away to correct pair
    for p in range(P):
        for w in range(W):
            ors = []
            for t in range(P):
                h, a = rb[t][w]
                ors.append(f"(and index_{p}_{w}_{t} (= home_{p}_{w} {h}) (= away_{p}_{w} {a}))")
            lines.append(f"(assert (or {' '.join(ors)}))")

    # Limit: a team must not appear more than twice in the same period slot across all weeks
    for t in range(1, n + 1):
        for p in range(P):
            terms = []
            for w in range(W):
                for i in range(P):
                    h, a = rb[i][w]
                    if h == t or a == t:
                        terms.append(f"(ite index_{p}_{w}_{i} 1 0)")
            if terms:
                lines.append(f"(assert (<= (+ {' '.join(terms)}) 2))")

    # Symmetry breaking: fix the first pair in the first slot to reduce equivalent solutions
    lines.append("(assert index_0_0_0)")

    # Request feasibility and model
    lines += ["(check-sat)", "(get-model)"]

    smtpath = os.path.join(smt_folder, fname)
    with open(smtpath, "w") as f:
        f.write("\n".join(lines))

### Parsing the Solver Output

This section handles the **post-processing** of the solver output from `sts_satisfability_n.smt2`. It has **no role in modeling or solving** the STS problem — it's purely a **software engineering task**.

If the solver returns `"sat"`, the subsequent lines define the values of decision variables. This part:

* Reads and groups lines defining variable assignments.
* Uses a regular expression to extract variable names and their integer values.
* Reconstructs the final schedule as a matrix of `[home, away]` pairs for each period and week.

This translation bridges the solver's raw output and the structured format needed for validation and reporting.

In [ ]:
def satisfability_run(n, solver, start_time):
    # Generate the SMT2 file name for the current instance size
    fname = f"sts_satisfability_{n}.smt2"

    # Generate the SMT-LIB encoding of the satisfiability problem for `n` teams
    satisfability_generate_smt2(n, fname)

    # Resolve the path to the smt2 file (assumes it's in the "smt" folder relative to the script)
    THIS_DIR = os.path.dirname(__file__)
    smt_folder = os.path.join(THIS_DIR, "smt")
    smtpath = os.path.join(smt_folder, fname)

    # Compute the remaining time allowed for this instance
    timeout_left = GLOBAL_TIMEOUT - (time.time() - start_time)

    # Call the solver (e.g., Z3 or CVC5) on the generated SMT2 file
    out, t1 = run_solver(solver, smtpath, timeout_left)

    # Parse the output lines
    lines = out.strip().splitlines()

    # If the solver returns "unsat", "timeout", or empty, return status with no solution
    if not lines or lines[0] != "sat":
        return (lines[0] if lines else "timeout"), None, t1

    # Otherwise, extract the variable assignments from the model
    blocks, cur = [], ""
    for L in lines[1:]:
        L = L.strip()
        if L.startswith("(define-fun"):
            if cur:
                blocks.append(cur)
            cur = L
        else:
            cur += " " + L
    blocks.append(cur)  # Add the last block

    # Regular expression to match integer variable definitions in SMT-LIB output
    pat = re.compile(r"\(define-fun\s+(\S+)\s*\(\)\s*Int\s+(-?\d+)\)")
    vals = {}
    for b in blocks:
        m = pat.match(b)
        if m:
            # Store the parsed variable values as integers
            vals[m.group(1)] = int(m.group(2))

    # Determine the number of periods (games per week) and weeks
    P, W = n // 2, n - 1

    # Construct the schedule solution: a matrix with shape [period][week]
    S0 = []
    for p in range(P):
        row = []
        for w in range(W):
            h = vals[f"home_{p}_{w}"]  # Home team
            a = vals[f"away_{p}_{w}"]  # Away team
            row.append([h, a])
        S0.append(row)

    # Return "sat" status, the solution matrix, and the solver runtime
    return "sat", S0, t1


### SMT-LIB Generation: Optimization Phase

This function encodes the **optimization phase**, where the goal is to rebalance a given valid schedule `S0` to minimize the home/away imbalance for each team.

The SMT-LIB file is built line-by-line and written to disk, just like in the satisfiability phase.

We start by initializing the logic (`QF_LIA`) and enabling model generation.

#### Decision Variables:

For each match slot `(p, w)`, we introduce:

* `flip_{p}_{w}`: a Boolean variable indicating whether the match is **flipped**, i.e., the home and away teams are swapped.

Using these flip variables, we define:

* `home_eff_{p}_{w}` and `away_eff_{p}_{w}`: the effective home and away teams after applying the flip. These are defined with `ite` expressions that conditionally reverse the original teams in `S0`.

#### Constraints:

We aim to keep the **home/away imbalance** bounded:

* For each team `t`, we compute how many times it appears as the effective home team (`H`) and away team (`A`).
* We then add two constraints:

  * The difference `H - A ≤ k`
  * The difference `A - H ≤ k`

These enforce that no team has an imbalance greater than `k`.

Finally, we ask the solver for a feasible solution (`check-sat`) and the corresponding model (`get-model`), and we write everything into the specified `.smt2` file.

In [ ]:
# ==== OPTIMIZATION ====

def optimization_generate_smt2(n, S0, k, fname):

    P, W = len(S0), len(S0[0])

    THIS_DIR = os.path.dirname(__file__)
    smt_folder = os.path.join(THIS_DIR, "smt")
    os.makedirs(smt_folder, exist_ok=True)

    # Initialize the SMT-LIB lines: set logic and request models
    lines = [
        "(set-logic QF_LIA)",
        "(set-option :produce-models true)"
    ]

    # Declare Boolean flip variables for each slot (period p, week w)
    for p in range(P):
        for w in range(W):
            lines.append(f"(declare-fun flip_{p}_{w} () Bool)")

    # Define the effective home and away team for each slot,
    # using an 'ite' expression that swaps home and away if the flip is true
    for p in range(P):
        for w in range(W):
            h0, a0 = S0[p][w]
            lines.append(f"(define-fun home_eff_{p}_{w} () Int (ite flip_{p}_{w} {a0} {h0}))")
            lines.append(f"(define-fun away_eff_{p}_{w} () Int (ite flip_{p}_{w} {h0} {a0}))")

    # For each team t, impose the imbalance constraints:
    # The difference (home games - away games) must be <= k,
    # and the opposite difference (away games - home games) must be <= k.
    for t in range(1, n + 1):
        H = " ".join(f"(ite (= home_eff_{p}_{w} {t}) 1 0)"
                    for p in range(P) for w in range(W))
        A = " ".join(f"(ite (= away_eff_{p}_{w} {t}) 1 0)"
                    for p in range(P) for w in range(W))
        # Bound imbalance
        lines.append(f"(assert (<= (- (+ {H}) (+ {A})) {k}))")
        lines.append(f"(assert (<= (- (+ {A}) (+ {H})) {k}))")

    # Request the solver to find a solution satisfying the constraints
    lines += ["(check-sat)", "(get-model)"]

    # Write the SMT-LIB code to the specified output file
    smtpath = os.path.join(smt_folder, fname)
    with open(smtpath, "w") as f:
        f.write("\n".join(lines))


### Optimization Loop: Minimizing Imbalance

This function runs the **optimization loop** that searches for the smallest possible imbalance `k` such that the given schedule `S0` can be rebalanced while satisfying all constraints.

We begin by computing the initial imbalance `k_hi` of the input solution `S0`. This defines the upper bound for `k`. Then, we apply a **binary search** in the range `[1, k_hi]` to progressively tighten the imbalance bound.

For each candidate value `k = mid`, we:

1. Generate the corresponding SMT-LIB file via `optimization_generate_smt2`.
2. Run the solver within the remaining time budget.
3. If the instance is satisfiable:

   * Parse the `flip_{p}_{w}` assignments from the model.
   * Apply the flips to the original `S0` to build a new (rebalanced) solution.
   * Update the current best solution and best `k`.
   * Stop early if `k = 1` (minimum possible imbalance).
4. If the instance is unsatisfiable, we increase the lower bound `low`.

At the end, we return the **best rebalanced schedule** found and the corresponding **minimum feasible imbalance** `k`.

In [ ]:
def optimization_run(n, S0, solver, start_time):

    k_hi = max(imbalance_of(S0, t) for t in range(1, n + 1))
    best, best_k = S0, k_hi
    low, high = 1, k_hi

    while low <= high and time.time() - start_time < GLOBAL_TIMEOUT:
        mid = (low + high) // 2
        timeout_left = GLOBAL_TIMEOUT - (time.time() - start_time)
        fname = f"sts_optimization_n{n}_k{mid}.smt2"
        optimization_generate_smt2(n, S0, mid, fname)

        THIS_DIR = os.path.dirname(__file__)
        smt_folder = os.path.join(THIS_DIR, "smt")
        smtpath = os.path.join(smt_folder, fname)

        out, _ = run_solver(solver, smtpath, timeout_left)
        out = out.strip()
        lines = out.splitlines()

        if lines and lines[0] == "sat":

            pattern = re.compile(r"flip_(\d+)_(\d+)[\s\S]*?(true|false)", re.MULTILINE)

            flips = {}
            matches = pattern.findall(out)
            for m in matches:
                p, w, val = int(m[0]), int(m[1]), (m[2] == "true")
                flips[(p, w)] = val

            P_, W_ = len(S0), len(S0[0])
            new = []
            for p in range(P_):
                row = []
                for w in range(W_):
                    h0, a0 = S0[p][w]
                    row.append([a0, h0] if flips.get((p, w), False) else [h0, a0])
                new.append(row)

            best, best_k = new, mid

            if best_k == 1:
                break

            high = mid - 1

        else:
            print(f"UNSAT for k = {mid}")
            low = mid + 1

    return best, best_k


### Main Execution Flow

This is the main entry point of the program. For each input size `n`:

1. **Satisfiability Phase**: Generates and solves the initial scheduling problem. If no solution is found (timeout/unsat), the result is saved and the function returns.
2. **Validation**: The generated schedule is validated using the provided checker.
3. **Optimization Phase**: If satisfiability succeeded, it runs a binary search to find the smallest possible imbalance `k` by flipping matches.
4. **Final Validation and Save**: The optimized schedule (if any) is validated and the result is saved to a `.json` file under `res/SMT/{n}.json`.

The `save_json` helper stores all relevant metrics: runtime, optimality, imbalance `k`, and the actual schedule. Command-line arguments allow choosing the solver, input sizes, and timeout.

In [ ]:
# ==== MAIN ====

def solve_and_save(n, solver):
    start_time = time.time()
    header = f"\n=== n = {n}, solver = {solver.upper()} ==="
    print(header)

    # === SATISFIABILITY PHASE ===
    # Run the satisfiability search and get initial solution S0
    status, S0, t1 = satisfability_run(n, solver, start_time)

    # If unsat or timeout, print and save result, then exit
    if status != "sat":
        if status == "timeout":
            print(f"TIMEOUT")
            save_json(n, solver, 300, None, 300, None, None, False)
        else:
            print(f"{status.upper()}")
        return

    # Validate the satisfiability solution
    result = check_solution(S0, None, t1, False)
    if result != 'Valid solution':
        print(f"Satisfiability checker FAILED: {result}")
        return

    # Check if time already exceeded
    elapsed = time.time() - start_time
    if elapsed >= GLOBAL_TIMEOUT:
        print(f"TIMEOUT")
        save_json(n, solver, t1, S0, t1, S0, None, False)
        return

    print(f"SATISFIABILITY COMPLETE: valid schedule in {t1:.2f}s")
    print(f"Starting Optimization...")

    # === OPTIMIZATION PHASE ===
    # Try to minimize imbalance k via binary search and flipping
    Sopt, k_opt = optimization_run(n, S0, solver, start_time)
    total = time.time() - start_time

    # Validate optimized solution (if any)
    if Sopt:
        result = check_solution(Sopt, k_opt, total, True)
        if result != 'Valid solution':
            print(f"Optimization checker FAILED: {result}")
            return

    # Print summary message
    if total >= GLOBAL_TIMEOUT:
        print(f"TIMEOUT during Optimization after {total:.2f}s")
    else:
        print(f"OPTIMIZATION COMPLETE: best imbalance k* = {k_opt} in {total:.2f}s")

    # Save result to JSON
    save_json(n, solver, t1, S0, total, Sopt, k_opt, total < GLOBAL_TIMEOUT)


def save_json(n, solver, t1, S0, total, Sopt, k_opt, optimal):
    # Prepare output folder
    folder = os.path.join(ROOT, "res", "SMT")
    os.makedirs(folder, exist_ok=True)
    outpath = f"{folder}/{n}.json"

    # Load existing results if any
    if os.path.exists(outpath):
        with open(outpath, "r") as f:
            result = json.load(f)
    else:
        result = {}

    # Format output data depending on whether optimization succeeded
    if Sopt is not None and k_opt is not None:
        final_sol = {
            "time": min(300, int(total)),  # Cap at 300s
            "optimal": optimal,            # Whether run finished in time
            "obj": k_opt,                  # Minimum imbalance k*
            "sol": Sopt                    # Optimized schedule
        }
    else:
        final_sol = {
            "time": 300,
            "optimal": False,
            "obj": None,
            "sol": []
        }

    # Save result under current solver key
    result[solver] = final_sol

    # Write to JSON file
    with open(outpath, "w") as f:
        json.dump(result, f, indent=2)

    print(f"Result saved: {outpath}")


# === ENTRY POINT ===
if __name__ == "__main__":
    p = argparse.ArgumentParser()

    # Choose which solver to use
    p.add_argument("--solver", choices=SOLVERS.keys(), default="z3",
                   help="Which SMT solver to use for both phases")

    # Provide list of team sizes to solve
    p.add_argument("--teams", type=int, nargs="+", default=[4,6,8,10,12,14,16,18,20,22,24],
                   help="List of teams to solve")

    # Global timeout across satisfiability + optimization
    p.add_argument("--time-limit", type=float, default=300.0,
                   help="Global timeout in seconds for solving (default: 300)")

    args = p.parse_args()
    GLOBAL_TIMEOUT = args.time_limit

    # Run solver for each input n
    start = time.time()
    for n in args.teams:
        solve_and_save(n, args.solver)


### Possibili domande e risposte

#### 1. **Perché avete introdotto le variabili `index_{p,w,t}`? Non bastava usare direttamente `home_{p,w}` e `away_{p,w}`?**

Le `index_{p,w,t}` rappresentano una **scelta booleana tra le partite candidate** in ciascuno slot `(p,w)`, prese dalla matrice `rb` generata con il round-robin.
Questo approccio:

* garantisce che le partite ammesse siano solo quelle previste dal round robin;
* consente di legare facilmente `home_{p,w}` e `away_{p,w}` alla partita effettivamente selezionata via un vincolo disgiuntivo;
* permette di aggiungere vincoli su conteggi e combinazioni (es. ogni coppia usata una sola volta) in modo semplice e scalabile.

---

#### 2. **Se `rb` contiene partite ben formate, perché aggiungete il vincolo “ogni team gioca una volta a settimana”?**

Perché `rb` è solo una **lista di opzioni**: non è detto che l’SMT ne selezioni un sottoinsieme che rispetta questo vincolo a meno che non lo **imponiamo esplicitamente**.
L’SMT potrebbe scegliere, ad esempio, di far giocare lo stesso team due volte nella stessa settimana, o di saltarne uno, se non vincoliamo la struttura globale della soluzione.

---

#### 3. **Come funziona esattamente l'ottimizzazione? Cos'è `flip_{p,w}`?**

Dopo aver trovato una soluzione valida, vogliamo **minimizzare lo sbilanciamento home/away**.
`flip_{p,w}` è una variabile booleana che, se vera, **inverte** i team home/away in quello slot.
L’ottimizzazione consiste nel trovare il valore minimo di `k` tale che, anche applicando flip, **nessun team gioca in casa o fuori più di `k` volte rispetto all’altro**.

---

#### 4. **Perché usate la ricerca binaria su `k` nella fase di ottimizzazione?**

Perché vogliamo trovare il **valore minimo possibile** di `k` che rende il problema ammissibile (cioè bilanciabile).
La ricerca binaria è efficiente perché ad ogni step dimezziamo lo spazio di ricerca e testiamo la fattibilità tramite SMT.
Se una soluzione esiste per `k = mid`, proviamo valori più piccoli; altrimenti aumentiamo il limite.

---

#### 5. Perché non avete usato il solving incrementale (push/pop) nell’ottimizzazione?

Avremmo potuto implementare la ricerca binaria su k usando push/pop per aggiungere progressivamente vincoli più stretti sull’imbalzo massimo consentito.
Tuttavia, abbiamo osservato che il vero collo di bottiglia del nostro pipeline è la fase di satisfiability, che richiede molto più tempo per produrre una soluzione valida.
L’ottimizzazione, al contrario, era costantemente veloce (spesso pochi secondi), e non giustificava la complessità aggiuntiva di una gestione incrementale.
Per questo abbiamo preferito mantenere l’ottimizzazione semplice e concentrare gli sforzi sull’efficienza e robustezza della prima fase.